## Intro
Information about this experiment can be found in the [markdown](./2024-10-21-try-llm-lowrop-holecleaning-packoff-with-debug.md).

## Set up environment

In [16]:
from dotenv import load_dotenv

_ = load_dotenv(override=True)

In [17]:
import os
from common.context import LLMTagPredictionContext

DEBUG = True
EXPERIMENT_ID = "2024-10-21-try-llm-holecleaning-debug"
RUN_ID = "1-holecleaning-gpt-4o-mini"

CONTEXT = LLMTagPredictionContext(
    description="Try asking LLM to assess all tags at once. Give examples.",
    experiment_id=EXPERIMENT_ID,
    run_id=RUN_ID,
    tags_in_scope=sorted(
        [
            "holecleaning"
        ]
    ),
    llm_model=os.environ["AZURE_OPENAI_DEPLOYMENT_ID"],
    with_notags=True,
)

## Fetch datasets

In [18]:
from common.datasets import load_input_dataset

dataset_df = load_input_dataset(
    "reviewed_distributed_ddr_v3.csv",
    columns_to_convert_to_sets=["tags", "Reviewed tags"],
)
dataset_df

,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN
...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\r\nMeanwhil...,INTERV,N,SAFETY,{},YES,{},NaN
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,"{lowrop, lostcirculation}",ROP 10m/hr.


## Apply the model

In [19]:
import pandas as pd
from common.llm import ask_openai

examples = dataset_df.sort_values(by="Text", key=lambda x: x.str.len())
examples = examples[examples["Text"].str.len() > 40]

examples = [
    examples[examples["Reviewed tags"].apply(lambda tags: tag in tags)][
        ["Text", "Reviewed tags"]
    ].head(2)
    for tag in CONTEXT.tags_in_scope
]

# Concatenate the examples into a single DataFrame
examples = pd.concat(examples, ignore_index=True)
# Convert 'Reviewed tags' to a sorted list
examples["Reviewed tags"] = examples["Reviewed tags"].apply(
    lambda tags: tuple(sorted(tags))
)
# Drop duplicate rows
examples = examples.drop_duplicates()
examples = examples.sort_values(by="Text")
examples

,Text,Reviewed tags
1,"""Circulated hole clean due to poor hole cleani...","(holecleaning,)"
0,Circulate due to excess cuttings at shakers.,"(holecleaning,)"


In [20]:
global DEBUG
SYSTEM_PROMPT = f"""
You will be given a description of a daily drilling report, and your task is to select which of these tags apply to this report.

Possible tags: {', '.join(CONTEXT.tags_in_scope)}

Respond only with the list of tags that apply to the report, separated by commas. If no tags apply, respond with `no tags`. Stick to the definition for the tag, do not make assumptions based on an context.

This is the definition of the tags:
- holecleaning: Ineffective removal of cuttings from the wellbore, potentially leading to stuck pipe, pack-off, tight hole or other issues.
The typical actions to improve this is to increase flowrate and/or increase drillstring rotations, reduce drilling speed, stop to circulate or backream (rotate while pulling out with pumps on).
If simulations on hole cleaning are mentioned, do not select this tag.
If hole cleaning is not specifically mentioned, do not select this tag.
Tag it only if you are sure that the hole cleaning is ineffective.

Other situations that might occur during drilling: Lost Circulation, Hard Drilling, Wellbore Stability, Stuck Pipe, Well Control, Low Rate of Penetration, Tight Hole, Boulders, Shallow Gas, Shallow Water, Wellbore Breathing, Pack-Off, Directional Control, High ROP, Downhole Equipment Failure, Wait, Surface Equipment Failure.These situations may or may not occur at the same time with Hole Cleaning.
"""

if not DEBUG:
    for idx, (_, row) in enumerate(examples.iterrows()):
        SYSTEM_PROMPT += f"\n## Example report {idx}\nText: {row['Text']}\n\n## Correct response\n{', '.join(row['Reviewed tags'])}\n"

if DEBUG:
    SYSTEM_PROMPT += "Before you provide the answer, explain the reason for selecting each tag or lack of it. The reason should be supplied at the end, following |debug| separator. Answer template: tag1, tag2, tag3|debug|reason1, reason2, reason3"

CONTEXT.llm_system_prompt = SYSTEM_PROMPT

print(SYSTEM_PROMPT)
print(len(SYSTEM_PROMPT))


You will be given a description of a daily drilling report, and your task is to select which of these tags apply to this report.

Possible tags: holecleaning

Respond only with the list of tags that apply to the report, separated by commas. If no tags apply, respond with `no tags`. Stick to the definition for the tag, do not make assumptions based on an context.

This is the definition of the tags:
- holecleaning: Ineffective removal of cuttings from the wellbore, potentially leading to stuck pipe, pack-off, tight hole or other issues.
The typical actions to improve this is to increase flowrate and/or increase drillstring rotations, reduce drilling speed, stop to circulate or backream (rotate while pulling out with pumps on).
If simulations on hole cleaning are mentioned, do not select this tag.
If hole cleaning is not specifically mentioned, do not select this tag.
Tag it only if you are sure that the hole cleaning is ineffective.

Other situations that might occur during drilling: L

In [21]:
TEMPERATURE = 0

CONTEXT.llm_temperature = TEMPERATURE

In [22]:
from concurrent.futures import ThreadPoolExecutor
import os
from tqdm.auto import tqdm


# Define a function to call ask_openai and get the predicted tags
def get_predicted_tags(text) -> tuple[set, str | None, str | None]:
    global DEBUG
    system_prompt = SYSTEM_PROMPT
   
    try:
        response = ask_openai(
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_OPENAI_KEY"],
            api_version=os.environ["AZURE_OPENAI_API_VERSION"],
            deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_ID"],
            system_prompt=system_prompt,
            prompt=text,
        )
    except Exception as e:
        if "content management policy. Please modify your prompt" in str(e):
            print(e)
            return [], None, None  # running into the content filter
        raise

    def normalize_tag(t):
        # sometimes model makes mistakes
        t = t.lower()
        if t.startswith("tags:"):
            t = t[len("tags:") :]
        t = t.strip().strip("()")
        return t
    
    # Debug mode logic: if enabled, append reasoning for each tag
    if DEBUG and "|debug|" in response:
        resonse_tags = response.split("|debug|")[0].strip().split(",")
        debug_info = response.split("|debug|")[1].strip()  # Extract reasoning part
        tags = set(normalize_tag(t) for t in resonse_tags).intersection(
        CONTEXT.tags_in_scope
    )
        return tags, response, debug_info
    else:
        tags = set(normalize_tag(t) for t in response.strip().split(",")).intersection(
        CONTEXT.tags_in_scope)
        return tags, response, None

assessed_df = dataset_df.copy()


def parallel_apply(df, func, num_threads: int):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(tqdm(executor.map(func, df["Text"]), total=len(df)))
    return results


assessed_df[["Predicted", "CompleteAnswer", "Debug"]] = parallel_apply(
     assessed_df, get_predicted_tags, num_threads=2
 )

assessed_df

100%|██████████| 1442/1442 [07:04<00:00,  3.40it/s]


,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments,Predicted,CompleteAnswer,Debug
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report mentions circulating ...,The report mentions circulating the hole and c...
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any ineffective re...
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not specifically...,The report does not specifically mention ineff...
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any ineffective re...
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any issues related...
...,...,...,...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\r\nMeanwhil...,INTERV,N,SAFETY,{},YES,{},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any issues related...
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any issues related...
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN,{},no tags|debug|There is no mention of ineffecti...,There is no mention of ineffective removal of ...
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,"{lowrop, lostcirculation}",ROP 10m/hr.,{},no tags|debug|There is no mention of ineffecti...,There is no mention of ineffective removal of ...


In [23]:
assessed_df.head()

,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments,Predicted,CompleteAnswer,Debug
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report mentions circulating ...,The report mentions circulating the hole and c...
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any ineffective re...
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not specifically...,The report does not specifically mention ineff...
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any ineffective re...
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN,{},no tags|debug|The report does not mention any ...,The report does not mention any issues related...


In [24]:
# nothing to do, DDR tagging using regex rules is already applied to the dataset in this experiment
from typing import Iterable
from common.assessment import expand_tags

assessed_df = expand_tags(
    assessed_df,
    tags_in_scope=CONTEXT.tags_in_scope,
    ground_truth_tags_column="Reviewed tags",
    predicted_tags_column="Predicted",
)
assessed_df

,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments,CompleteAnswer,Debug,expected__holecleaning,actual__holecleaning
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report mentions circulating ...,The report mentions circulating the hole and c...,False,False
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not mention any ...,The report does not mention any ineffective re...,False,False
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not specifically...,The report does not specifically mention ineff...,False,False
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not mention any ...,The report does not mention any ineffective re...,False,False
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not mention any ...,The report does not mention any issues related...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\r\nMeanwhil...,INTERV,N,SAFETY,{},YES,{},NaN,no tags|debug|The report does not mention any ...,The report does not mention any issues related...,False,False
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN,no tags|debug|The report does not mention any ...,The report does not mention any issues related...,False,False
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN,no tags|debug|There is no mention of ineffecti...,There is no mention of ineffective removal of ...,False,False
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,"{lowrop, lostcirculation}",ROP 10m/hr.,no tags|debug|There is no mention of ineffecti...,There is no mention of ineffective removal of ...,False,False


In [25]:
from common.datasets import save_assessed_dataset

save_assessed_dataset(
    assessed_df,
    context=CONTEXT,
    experiment_id=EXPERIMENT_ID,
    run_id=RUN_ID,
)

2_assessed_datasets\2024-10-21-try-llm-holecleaning-debug-1-holecleaning-gpt-4o-mini


## Evaluate predicted tags

In [26]:
from common.evaluation import TagMatchingEvaluator

evaluator = TagMatchingEvaluator(
    assessed_df=assessed_df,
    tags_in_scope=CONTEXT.tags_in_scope,
    with_notags=CONTEXT.with_notags,
)

In [27]:
evaluator.eval_individual_ddrs()

,reviewed_distributed_ddr_v2.csv,Text,phase,code,subCode,tags,Are tags correct?,Reviewed tags,Comments,CompleteAnswer,Debug,expected__holecleaning,actual__holecleaning,expected__notags,actual__notags,precision,recall,f1,true_positives
0,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report mentions circulating ...,The report mentions circulating the hole and c...,False,False,True,True,1.0,1.0,1.0,1
1,a1f86f80-135e-458b-aafc-3af30d2476f2_main_61b0...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not mention any ...,The report does not mention any ineffective re...,False,False,True,True,1.0,1.0,1.0,1
2,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,Circulated hole with reduced flow due to sand ...,INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not specifically...,The report does not specifically mention ineff...,False,False,True,True,1.0,1.0,1.0,1
3,a1f86f80-135e-458b-aafc-3af30d2476f2_main_d1ce...,"Circulated hole (4400 lpm, 70 rpm, 2-4 kNm) wh...",INTCSG1,N,CIR,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not mention any ...,The report does not mention any ineffective re...,False,False,True,True,1.0,1.0,1.0,1
4,a1f86f80-135e-458b-aafc-3af30d2476f2_main_e6c0...,Laid down cement head. Moved rig to J-3 slot....,SURF,P,LD,{shallowwater},YES,{shallowwater},NaN,no tags|debug|The report does not mention any ...,The report does not mention any issues related...,False,False,True,True,1.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,a1f86f80-135e-458b-aafc-3af30d2476f2_main_7965...,Meeting with onshore forward plan.\r\nMeanwhil...,INTERV,N,SAFETY,{},YES,{},NaN,no tags|debug|The report does not mention any ...,The report does not mention any issues related...,False,False,True,True,1.0,1.0,1.0,1
1438,a1f86f80-135e-458b-aafc-3af30d2476f2_main_fc63...,Drilled from 3903 m to 3921 m. - WOB = 6 - ...,RES1,P,DRL,{},YES,{},NaN,no tags|debug|The report does not mention any ...,The report does not mention any issues related...,False,False,True,True,1.0,1.0,1.0,1
1439,a1f86f80-135e-458b-aafc-3af30d2476f2_main_12d5...,CIRC OUT FILL W/ 80 SPM/2000 PSI.,PROD1,C,NaN,{},YES,{},NaN,no tags|debug|There is no mention of ineffecti...,There is no mention of ineffective removal of ...,False,False,True,True,1.0,1.0,1.0,1
1440,a1f86f80-135e-458b-aafc-3af30d2476f2_main_644e...,"Drilled 12-1/4"" hole from 3916m to 3931m with...",INT2,P,DRLDIR,{},NO,"{lowrop, lostcirculation}",ROP 10m/hr.,no tags|debug|There is no mention of ineffecti...,There is no mention of ineffective removal of ...,False,False,True,True,1.0,1.0,1.0,1


In [28]:
evaluator.eval_per_tag()

,tag,precision,recall,f1,true_positives,positives_in_ground_truth,negatives_in_ground_truth
0,holecleaning,0.780488,0.888889,0.831169,32,36,1406


In [29]:
evaluator.average_metrics()

,Type,precision,recall,f1
0,Average per DDR,0.990985,0.990985,0.990985
1,Average per Tag,0.780488,0.888889,0.831169


## Save evaluation report

In [30]:
from common.datasets import save_evaluation_report

save_evaluation_report(
    experiment_id=EXPERIMENT_ID,
    run_id=RUN_ID,
    dataset_df=dataset_df,
    assessed_df=assessed_df,
    evaluator=evaluator,
    context=CONTEXT,
)